# Credit Card Fraud Analysis with Homomorphic Encryption

This notebook demonstrates a secure statistical analysis workflow using the **LattigoStats** library. 
We use Homomorphic Encryption (HE) to perform computations on encrypted credit card transaction data without ever decrypting it.

### Secure Workflow:
1. **KeyGen**: Data Decryption and Inspection Authority (DDIA) generates cryptographic keys.
2. **Encryption**: The Data Provider encrypts the dataset using the public key.
3. **Computation**: The Data Analyst (DA) runs statistical jobs on the encrypted data.
4. **Decryption**: The DDIA decrypts the results using the secret key.
5. **Inspection**: The DDIA verifies that the results do not violate privacy policies (e.g., k-anonymity).

In [ ]:
from lattigo_stats import LattigoStats
import os

# Initialize the helper
ls = LattigoStats(bin_dir="./bin")

## 1. Build Secure Tools
Ensure the Go binaries are compiled and ready.

In [ ]:
!go build -o bin/ddia ./cmd/ddia
!go build -o bin/do_encrypt ./cmd/do_encrypt
!go build -o bin/da_run ./cmd/da_run

## 2. Key Generation
Generate CKKS parameters and keys. Profile B supports bootstrapping for deeper computations.

In [ ]:
ls.keygen(profile="B", output_dir="./keys")

## 3. Data Preparation & Encryption
Define the data schema and encrypt the dataset.

In [ ]:
columns = [
    {"name": "Time", "type": "numerical", "description": "Transaction time"},
    *[{"name": f"V{i}", "type": "numerical"} for i in range(1, 29)],
    {"name": "Amount", "type": "numerical", "description": "Transaction amount"},
    {"name": "Class", "type": "categorical", "category_count": 2, "description": "Target: 1 for fraud, 0 otherwise"}
]

ls.generate_schema("credit_card_transactions", columns, "schema.json")

In [ ]:
# Ensure you have the cleaned_creditcard.csv in the root
ls.encrypt("cleaned_creditcard.csv", "schema.json", "./keys/public.key", "./encrypted", profile="B")

## 4. Run Statistical Analysis on Encrypted Data
Compute the average transaction amount for fraudulent transactions (`Class == 1`).

In [ ]:
conditions = [{"column": "Class", "value": 1}]
ls.generate_job("fraud_analysis_v1", "ba", "credit_card_transactions", "Amount", conditions, "job_ba.json")

ls.run_job("job_ba.json", "./encrypted", "./keys", "result.ct")

## 5. Decryption & Visualization
Decrypt the result and visualize the output.

In [ ]:
ls.decrypt("./keys/secret.key", "result.ct/result.ct", "decrypted_result.json", profile="B")

values = ls.load_decrypted_result("decrypted_result.json")

# Note: If the result shows large numbers (e.g. 1e160), it likely means 
# the filter (Class == 1) matched 0 rows, leading to a division by zero error in HE.
print(f"First 5 decrypted values: {values[:5]}")
ls.plot_results(values, title="Bin-Average Result (Fraudulent Transactions)")

## 6. Privacy Inspection
Verify that the result is safe to release.

In [ ]:
ls.inspect("decrypted_result.json")